In [6]:
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap


## Cargo Dataset principal

In [7]:
df = pd.read_csv('../../data/Top100_prediction_con.csv')

In [8]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'NYC_2') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'income', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['income'], ts_predict.drop(columns=['income'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(random_state=106,
                                        max_depth=10,
                                        min_samples_leaf=2,
                                        min_samples_split=5,
                                        n_estimators=50),
        lags=7
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)

    start_date = '2016-04-25' 
    end_date ='2016-05-30'
    # Crear un rango de fechas
    date_range = pd.date_range(start=start_date, end=end_date)
    # Crear el dataframe
    df1 = pd.DataFrame(date_range, columns=['date'])
    df1['date'] = pd.to_datetime(df1['date'])
    df1['weekday'] = df1['date'].dt.day_name()
    df1['event'] = 0
    df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
    df1.sort_values('date', inplace=True, ascending=True)
    df1.set_index('date', inplace=True)
    df1 = df1.asfreq('D')
    exog_test = pd.concat([exog_test, df1], ignore_index=False)
    
    # Predecir
    predicciones = forecaster.predict(steps=60, exog=exog_test)
    
    # Almacenar resultados
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    #print(f"Predicciones generadas para el producto: {producto}")


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider 

In [9]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediccion'] = tiendas_inventario['prediccion'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['producto','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,producto,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,362.5314,532.0,569.0
1,HOME_&_GARDEN_1_027,298.5000,532.0,526.0
2,HOME_&_GARDEN_1_053,2095.8000,1268.0,1234.0
3,HOME_&_GARDEN_1_140,935.6250,743.0,704.0
4,HOME_&_GARDEN_1_177,477.9375,833.0,763.0
...,...,...,...,...
95,SUPERMARKET_3_499,1050.6240,490.0,473.0
96,ACCESORIES_1_158,1742.1138,1179.0,1174.0
97,SUPERMARKET_3_282,1812.4800,1249.0,1228.0
98,ACCESORIES_1_354,2475.6354,1564.0,1521.0


In [10]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

94564.60230000001
71948.0
22616.602300000013


In [4]:
tienda =  pd.DataFrame(resultados)
file_name = "NYC_2_ventas.xlsx"
tienda.to_excel(file_name)

In [8]:
pd.DataFrame(resultados)

,producto,prediccion,test
0,ACCESORIES_1_108,"[1.2136363636363638, 0.6488095238095238, 0.435...","[0, 1, 0, 1, 2, 3, 0, 0, 0, 2, 0, 0, 0, 2, 0, ..."
1,HOME_&_GARDEN_1_027,"[1.213047619047619, 3.167738095238095, 2.104, ...","[3, 3, 0, 1, 2, 0, 0, 0, 3, 3, 0, 0, 0, 1, 3, ..."
2,HOME_&_GARDEN_1_053,"[3.1244999999999994, 2.679238095238095, 1.6030...","[1, 8, 4, 2, 2, 5, 1, 12, 6, 2, 4, 0, 1, 4, 3,..."
3,HOME_&_GARDEN_1_140,"[3.146396825396825, 0.9730952380952382, 1.0827...","[1, 5, 1, 1, 3, 1, 0, 5, 2, 0, 1, 1, 1, 3, 1, ..."
4,HOME_&_GARDEN_1_177,"[5.475333333333333, 5.998904761904763, 3.36914...","[3, 7, 7, 3, 3, 2, 5, 6, 6, 5, 6, 1, 0, 0, 0, ..."
...,...,...,...
95,SUPERMARKET_3_499,"[7.395714285714287, 7.452857142857142, 4.83038...","[6, 0, 0, 5, 5, 9, 15, 22, 23, 20, 25, 25, 16,..."
96,ACCESORIES_1_158,"[1.3385238095238097, 1.5138571428571428, 1.327...","[2, 1, 3, 0, 1, 0, 0, 2, 6, 4, 2, 1, 2, 3, 5, ..."
97,SUPERMARKET_3_282,"[17.62847619047619, 18.95012698412698, 16.6286...","[35, 32, 16, 13, 21, 21, 33, 38, 52, 19, 20, 2..."
98,ACCESORIES_1_354,"[1.7014444444444448, 3.119761904761905, 1.6083...","[2, 2, 2, 5, 6, 2, 2, 5, 2, 1, 0, 1, 1, 7, 5, ..."


## Predigo 30 dias(los de test) 